In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

from ntk import Ntk
ntk = Ntk()
import pandas as pd

ntk.get_gender("Eirik")
#AssumedGender(input_name='Eirik', gender='mann', mode='list_lookup', assumed_name='Eirik')

input_navn, kjonn, metode, antatt_navn =ntk.get_gender("Eirik")

ntk.get_gender("Ulerikka")

ntk.get_gender(" 'Lan Marie Nguyen Berg'")

ntk.get_gender("Abracus Bakkus Casparus Daniel Berg'")

# Tving gjennom ML prediksjon: (returnerer 'M'/'F' som str )
ntk.predict_gender(" 'Lan Marie Nguyen Berg'")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AssumedGender(input_name='Eirik', gender='mann', mode='list_lookup', assumed_name='Eirik')

AssumedGender(input_name='Ulerikka', gender='kvinne', mode='predictor', assumed_name='Ulerikka')

AssumedGender(input_name=" 'Lan Marie Nguyen Berg'", gender='kvinne', mode='list_lookup', assumed_name='Lan')

AssumedGender(input_name="Abracus Bakkus Casparus Daniel Berg'", gender='mann', mode='list_lookup', assumed_name='Abracus')

'M'

In [2]:
ntk.get_gender("Eirik")[1]

'mann'

In [3]:
# Har navnedag
url = "https://no.wikipedia.org/wiki/Liste_over_norske_navnedager"
data = pd.read_html(url, header=0, index_col=0)[0]
#data.head()
data = data[["Navn"]]
data.dropna(subset=['Navn'], how='all', inplace = True) # skip 1.jan, er nan
data.head()

# | Navn  | dato
# | Eirik | 18 mai
har_navnedag = []
har_navnedag.append(["Jesus", "1. januar"])

rene_navn1 = []

for d in data.iterrows():
    #print(d[0], type(d[0]))
    #print(d[1].Navn)
    navn_pr_dag = [n for n in d[1].Navn.replace(",","").split() if n.lower() not in ["og", "(navn)"]]
    for n in navn_pr_dag:
        har_navnedag.append([n, d[0]])
        rene_navn1.append(n)
    #print()

df = pd.DataFrame.from_records(har_navnedag, columns=["Navn", "Dato"])
# save only this one scrape
#df.to_csv(path_or_buf="praenomen2genus/data/har_norsk_navnedag.csv", sep='\t', encoding='utf-8', index=False)

df.set_index("Navn", inplace=True)
#df.head()


#    !

# pokker finnes enda en liste:
url2 = "https://no.wikipedia.org/wiki/Liste_over_norske_navnedager_sortert_etter_navn"
data2 = pd.read_html(url2, header=0, index_col=0)[0]
data2 = data2[["Navnedag"]]
data2.columns = [["Dato"]]
#data2.head()
#data.describe()
rene_navn2 = []
for nn in data2.iterrows():
    rene_navn2.append(nn[0])

bare_i_forste = [n for n in rene_navn1 if n not in rene_navn2]
bare_i_andre = [n for n in rene_navn2 if n not in rene_navn1]

#print(len(bare_i_forste), bare_i_forste)
#print(len(bare_i_andre), bare_i_andre)
print("jeg burde ha så mange navn:", len(set(rene_navn1+rene_navn2)))


# la oss merge disse

#result = df.append(data2) #ignore_index=True # blir 1662 rader, virker for mye...
#result.describe()
#result

result2 = pd.merge(df, data2, how="outer", right_index=True, left_index=True) # 


,Navn
Dato,
2. januar,Dagfinn og Dagfrid
3. januar,Alfred og Alf og Alva
4. januar,Roar og Roger
5. januar,Hanna og Hanne
6. januar,"Aslaug, Åslaug og Aisha"


jeg burde ha så mange navn: 855


In [4]:
def test_navendager():
    nd_df = pd.read_csv("data/har_norsk_navnedag.csv", sep="\t")
    for n in nd_df.itertuples():
        print(n.Navn)
        # ah føkk. mangler kjønn på disse.

#test_navendager()

In [5]:
begge_kjonn = ["Gunnleiv", "Kim"]
kvinne = ["Hallgjerd"]
menn = ["Klement", "Clement", "Clément", "Salve"]

har_navnedag_kjonn = []

In [6]:
# norrøne navn
norr_url = "https://no.wikipedia.org/wiki/Liste_over_personer_i_norr%C3%B8n_mytologi"

norr = pd.read_html(norr_url, header=0, index_col=0)[0]
norr2 = norr[["Norrønt navn"]]

norr2.head()
#norr2.to_csv(path_or_buf="praenomen2genus/data/personer_i_norron_mytologi.csv", sep='\t', encoding='utf-8')#
personer_i_norron_mytologi = norr2

,Norrønt navn
Navn,
Allvis,"Allvís, «den som vet alt»"
Andrimne,"Andrimner, «en med sot i ansiktet»"
Andvare,Andvari
Angerboda,"Angrboða, «sorgvarsleren»"
Aurboda,Eirboða


In [7]:
# Bibelske navn
url1 = "https://www.ssb.no/a/magasinet/blandet/tab-2006-02-09-02.html" # jenter
url2 = "https://www.ssb.no/a/magasinet/blandet/tab-2006-02-09-01.html" # gutter

bibjent = pd.read_html(url1, index_col=0)[0] #  header=0, føkket det til..
print("bibjent", bibjent)
#bibjent.columns = [["Kommentar"]]
bibjent.index.names = ['Navn']
bibjent.head()

bibeljenter = ['Abiah', 'Abigail', 'Abijah', 'Abilene', 'Abishag', 'Abital', 'Adah', 'Adina', 'Apphia', 'Ariel', 'Asenath', 'Ashtoreth', 'Atarah', 'Azubah', 'Basemath', 'Bashemath', 'Basmath', 'Bathsheba', 'Bernice', 'Bethany', 'Bethel', 'Beulah', 'Bilhah', 'Bithiah', 'Candace', 'Carmel', 'Chloe', 'Damaris', 'Deborah', 'Delilah', 'Dinah', 'Dorcas', 'Drusilla', 'Eden', 'Edna', 'Elisabeth', 'Elisheba', 'Elizabeth', 'Esther', 'Eunice', 'Eve', 'Gethsemane', 'Hadassah', 'Hagar', 'Hannah', 'Havilah', 'Helah', 'Hephzibah', 'Hepzibah', 'Hulda', 'Huldah', 'Ivah', 'Jael', 'Jedidah', 'Jemima', 'Jerusha', 'Jezebel', 'Joanna', 'Jordan', 'Judith', 'Julia', 'Keren-happuch', 'Keturah', 'Keziah', 'Leah', 'Lois', 'Lydia', 'Magdalene', 'Mahalath', 'Mahlah', 'Mara', 'Martha', 'Mary', 'Mehetabel', 'Mehitabel', 'Micaiah', 'Michal', 'Miriam', 'Moriah', 'Naamah', 'Naomi', 'Neriah', 'Noa', 'Noah', 'Ophrah', 'Orpah', 'Peninnah', 'Persis', 'Philadelphia', 'Phoebe', 'Prisca', 'Priscilla', 'Rachel', 'Rahab', 'Rebecca', 'Rebekah', 'Rhoda', 'Ruth', 'Salome', 'Sapphira', 'Sarah', 'Sarai', 'Sela', 'Sharon', 'Sherah', 'Shiphrah', 'Susanna', 'Susannah', 'Syntyche', 'Tabitha', 'Talitha', 'Tamar', 'Tirzah', 'Tryphena', 'Tryphosa', 'Vashti', 'Zillah', 'Zilpah', 'Zipporah']
for j in bibeljenter:
    if j not in bibjent.index:
        bibjent.loc[j] = pd.Series({'Kommentar': "fra liste"})

bibjent["Kjønn"] = "f"



bibgutt = pd.read_html(url2, header=0, index_col=0)[0]
bibgutt.columns = [["Kommentar"]]
bibgutt.index.names = ['Navn']
bibgutt["Kjønn"] = "m"
bibgutt.head()

bibjent                                       1
0                                      
NaN  A-Å | Hjelp | Spørsmål | Kontakter


,1
Navn,
NaN,A-Å | Hjelp | Spørsmål | Kontakter


,Kommentar,Kjønn
Navn,,


In [8]:
# Jødiske navn

In [9]:
# Populære navn (periodisk oppdatet fra ssb?)
# for mange land her: http://www.uib.no/lle/23822/navnestatistikk-fra-mange-land

In [10]:
# Som rimer på
# som likner på
# som inneholder en "o"
# som besteforeldrenes generasjon brukte
# som olderforeldrenes generasjon brukte
# som tip-tip-tip...


In [11]:
import os
import random
import logging
import pickle
from nltk import NaiveBayesClassifier, classify

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger


# dir = os.path.dirname('../')
# print(dir)
pickle_file = os.path.join('../../', 'data/no_names.pickle')
print("file:", pickle_file)

#pickle_file = 'data/no_names.pickle'
#%pwd
#os.getcwd()

<Logger __main__ (INFO)>

file: ../../data/no_names.pickle


In [12]:

# this is just the predictor
class genderPredictor():
    """ This is the AI model that kicks in if name is not found in lists.
    It is rewritten to just load picke file with names data.
    New names data ca be generated with aux_picklemaker_NOLoader.py
    """

    def getFeatures(self):
        maleNames, femaleNames = self._loadNames()

        featureset = list()
        for nameTuple in maleNames:
            features = self._nameFeatures(nameTuple[0])
            featureset.append((features, 'M'))

        for nameTuple in femaleNames:
            features = self._nameFeatures(nameTuple[0])
            featureset.append((features, 'F'))
        #print(featureset)
        # [({'last_letter': 'S', 'last_two': 'AS', 'last_three': 'EAS', 'last_is_vowel': False}, 'M'), ({'last_letter': 'Z', 'last_two': 'SZ', 'last_three': 'USZ', 'last_is_vowel': False}, 'M'), ({'last_letter': 'R', 'last_two': 'UR', 'last_three': 'VUR', 'last_is_vowel': False}, 'M'), ({'last_letter': 'S', 'last_two': 'AS', 'last_three': 'BAS', 'last_is_vowel': False}, 'M'), ({'last_letter': 'R', 'last_two': 'UR', 'last_three': 'DUR', 'last_is_vowel': False}, 'M'), 
        return featureset

    def trainAndTest(self, trainingPercent=0.80):
        featureset = self.getFeatures()
        random.shuffle(featureset)
        name_count = len(featureset)
        cut_point = int(name_count*trainingPercent)
        train_set = featureset[:cut_point]
        test_set = featureset[cut_point:]

        self.train(train_set)
        logger.info("Accuracy: %s" % (self.test(test_set)))
        return self.test(test_set)

    def classify(self, name):
        feats = self._nameFeatures(name)
        return self.classifier.classify(feats)

    def train(self, train_set):
        self.classifier = NaiveBayesClassifier.train(train_set)
        return self.classifier

    def test(self, test_set):
        return classify.accuracy(self.classifier, test_set)

    def getMostInformativeFeatures(self, n=15):
        logger.info(self.classifier.show_most_informative_features(n))
        return self.classifier.most_informative_features(n)

    def _loadNames(self):
        # print USSSALoader.getNameList()
        f = open(pickle_file, 'rb')
        names = pickle.load(f)
        # return NOLoader.getNameList()
        # print names
        return names

    def _nameFeatures(self, name):
        name = name.upper()
        return {
            'last_letter': name[-1],    # -a er oftere for jenter
            'last_two': name[-2:],      # -en er oftere for gutter
            'last_three': name[-3:],    # -ine er oftere for jenter..
            'last_is_vowel': (name[-1] in u'AEIOUYÆØÅ')  # slutter noen navn på
            # ÆØÅ i det heletatt? (øker accuracy til 0.82, men senker
            # mini-testen for herrer med 7 prosentpoeng og øker damer med 2
            # prosentpoeng)
        }

%pwd
noe = genderPredictor()
noe.trainAndTest()

#noe.getMostInformativeFeatures()
#noe.classify("Noe")

'/Users/eirik.stavelin/Projects/ntk/tests/explorations'

INFO:__main__:Accuracy: 0.8678899082568807


0.8678899082568807

In [13]:
# fra Roberts kode:
import numpy as np
# def featureEngineer(df):
#     output = df
    
#     # Day of week. Monday = 0, Sunday = 6
#     output['DayCreated'] = output['CreatedAt'].dt.dayofweek

#     # Weekend or not. Weekend = 1
#     output['Weekend'] = np.where(((output['DayCreated']==5) | (output['DayCreated']==5)),1,0)

#     global categorical
#     categorical = ['Age','Gender','Year','DayCreated','Weekend']

#     dummies = output[categorical].astype(str)
#     dummies = dummies.fillna('MissingValue')
#     dummies = pd.get_dummies(dummies)

#     output = pd.concat([output,dummies], axis=1, join='inner')    

#     output['Body'] = output['Body'].astype(str)

#     return output

# data = featureEngineer(data)
